In [1]:
import xarray
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
import math

from Preprocessing import *

prep = Preprocessing()

In [12]:
from Preprocessing import *
merged_hornsea = pd.read_parquet("preprocessed_hornsea_with_energy.parquet")
merged_pes = pd.read_parquet("preprocessed_pes_with_energy.parquet")
merged_demand = pd.read_parquet("preprocessed_demand_with_energy.parquet")

In [1]:
from Preprocessing import *

prep = Preprocessing()

# merged_hornsea = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_hornsea":"nc_files", "ncep_gfs_hornsea":"nc_files"},
#                                               deployment = False, merge_with_outage_data = True, json_file_path = "nc_files/REMIT", energy_data_dict = {"Energy_data":"csv_files"},
#                                               non_numerical_columns = ["unavailabilityType", "affectedUnit"],
#                                               fft = False, )

# merged_pes = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_pes":"nc_files", "ncep_gfs_pes":"nc_files"},
#           deployment = False, merge_with_outage_data = False, energy_data_dict = {"Energy_data":"csv_files"},
#                                               non_numerical_columns = ["unavailabilityType", "affectedUnit"],
#                                               fft = False, )

# merged_demand = prep.perform_preprocessing_pipeline(geo_data_dict = {"dwd_icon_eu_demand":"nc_files", "ncep_gfs_demand":"nc_files"},
#           deployment = False, merge_with_outage_data = True, energy_data_dict = {"Energy_data":"csv_files"},
#                                               non_numerical_columns = ["unavailabilityType", "affectedUnit"],
#                                               fft = False, )

Perform data cleaning on the weather data...
Merge weather stations...
Merge with label data...
Preprocessing done!


In [13]:
import importlib
from Preprocessing import *

feature_engineerer_wind = FeatureEngineerer(label = 'Wind_MWh_credit', columns_to_ohe = ['unavailabilityType', 'affectedUnit', "outage"])
feature_engineerer_wind.perform_feature_engineering(merged_hornsea, deployment = False)

feature_engineerer_solar = FeatureEngineerer(label = "Solar_MWh_credit")
feature_engineerer_solar.perform_feature_engineering(merged_pes, deployment = False)

merged_pes_simple = merged_pes[['solar_down_rad', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_solar_baseline = FeatureEngineerer(label = 'Solar_MWh_credit')
feature_engineerer_solar.perform_feature_engineering(merged_pes_simple, deployment = False)

merged_hornsea_simple = merged_hornsea[['wind_speed_100', 'Solar_MWh_credit', 'Wind_MWh_credit']]
feature_engineerer_wind_baseline = FeatureEngineerer(label = 'Wind_MWh_credit')
feature_engineerer_wind_baseline.perform_feature_engineering(merged_hornsea_simple, deployment = False)

### __Wind Energy Forecast__

__baseline__

In [12]:
import model_utils
import importlib
importlib.reload(model_utils)

quantiles = np.arange(0.1, 1.0, 0.1)

# Specify model save directory
model_save_dir_qr = "qr_model_wind"

qr_model = model_utils.QuantileRegressorModel(feature_engineerer_wind_baseline, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
qr_model.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {qr_model.pinball_score()}")

Saved Quantile Regressor model for quantile 0.1 to qr_model_wind\qr_model_quantile_0.1.pkl
Saved Quantile Regressor model for quantile 0.2 to qr_model_wind\qr_model_quantile_0.2.pkl
Saved Quantile Regressor model for quantile 0.30000000000000004 to qr_model_wind\qr_model_quantile_0.30000000000000004.pkl
Saved Quantile Regressor model for quantile 0.4 to qr_model_wind\qr_model_quantile_0.4.pkl
Saved Quantile Regressor model for quantile 0.5 to qr_model_wind\qr_model_quantile_0.5.pkl
Saved Quantile Regressor model for quantile 0.6 to qr_model_wind\qr_model_quantile_0.6.pkl
Saved Quantile Regressor model for quantile 0.7000000000000001 to qr_model_wind\qr_model_quantile_0.7000000000000001.pkl
Saved Quantile Regressor model for quantile 0.8 to qr_model_wind\qr_model_quantile_0.8.pkl
Saved Quantile Regressor model for quantile 0.9 to qr_model_wind\qr_model_quantile_0.9.pkl
Quantile Regressor Pinball Score: 53.08909151412246


__xgboost__

In [42]:
import model_utils


quantiles = np.arange(0.1, 1.0, 0.1)

model_save_dir_xgboost = "xgboost_model_wind"

hyperparams = {"objective": "reg:quantileerror",
            "tree_method": "hist",
            "quantile_alpha": quantiles,
            "learning_rate": 0.01,
            "max_depth": 10}

xgboost_model_wind = model_utils.XGBoostModel(feature_engineerer_wind, quantiles=quantiles, model_save_dir=model_save_dir_xgboost, load_pretrained=False, 
                                              hyperparams = hyperparams
                                              )
xgboost_model_wind.train_and_predict()  # This will skip training if the model is already loaded
print(f"XGBoost Pinball Score: {xgboost_model_wind.pinball_score()}")

[0]	Train-quantile:99.03586	Val-quantile:90.84414
[1]	Train-quantile:98.92821	Val-quantile:90.76461
[2]	Train-quantile:98.82061	Val-quantile:90.68514
[3]	Train-quantile:98.71315	Val-quantile:90.60492
[4]	Train-quantile:98.60570	Val-quantile:90.52455
[5]	Train-quantile:98.49836	Val-quantile:90.44456
[6]	Train-quantile:98.39124	Val-quantile:90.36449
[7]	Train-quantile:98.28420	Val-quantile:90.28433
[8]	Train-quantile:98.17746	Val-quantile:90.20423
[9]	Train-quantile:98.07064	Val-quantile:90.12410
[10]	Train-quantile:97.96416	Val-quantile:90.04424
[11]	Train-quantile:97.85772	Val-quantile:89.96434
[12]	Train-quantile:97.75142	Val-quantile:89.88439
[13]	Train-quantile:97.64515	Val-quantile:89.80467
[14]	Train-quantile:97.53909	Val-quantile:89.72525
[15]	Train-quantile:97.43309	Val-quantile:89.64584
[16]	Train-quantile:97.32716	Val-quantile:89.56652
[17]	Train-quantile:97.22131	Val-quantile:89.48720
[18]	Train-quantile:97.11561	Val-quantile:89.40807
[19]	Train-quantile:97.01011	Val-quantile

### __Solar Energy Forecast__

__baseline modell__

In [7]:
quantiles = np.arange(0.1, 1.0, 0.1)
import model_utils

# Specify model save directory
model_save_dir_qr = "qr_model_solar"

qr_model_solar = model_utils.QuantileRegressorModel(feature_engineerer_solar_baseline, quantiles, model_save_dir=model_save_dir_qr, load_pretrained=False)
qr_model_solar.train_and_predict()  # This will skip training for already loaded models
print(f"Quantile Regressor Pinball Score: {qr_model_solar.pinball_score()}")

: 

__xgboost__

In [41]:
import model_utils
quantiles = np.arange(0.1, 1.0, 0.1)

model_save_dir_xgboost = "xgboost_model_solar"

hyperparams = {"objective": "reg:quantileerror",
            "quantile_alpha": quantiles,
            "learning_rate": 0.05,
            "max_depth": 10
            }

xgboost_model_solar = model_utils.XGBoostModel(feature_engineerer_solar, quantiles=quantiles, 
                                               hyperparams = hyperparams, 
                                               model_save_dir=model_save_dir_xgboost, load_pretrained=False)
xgboost_model_solar.train_and_predict()  # This will skip training if the model is already loaded
print(f"XGBoost Pinball Score: {xgboost_model_solar.pinball_score()}")

[0]	Train-quantile:43.78358	Val-quantile:91.30714
[1]	Train-quantile:41.57380	Val-quantile:86.09397
[2]	Train-quantile:39.51613	Val-quantile:81.25100
[3]	Train-quantile:37.58369	Val-quantile:76.66185
[4]	Train-quantile:35.76655	Val-quantile:72.35495
[5]	Train-quantile:34.06246	Val-quantile:68.31612
[6]	Train-quantile:32.45966	Val-quantile:64.48859
[7]	Train-quantile:30.95068	Val-quantile:60.88981
[8]	Train-quantile:29.53092	Val-quantile:57.52676
[9]	Train-quantile:28.19412	Val-quantile:54.38676
[10]	Train-quantile:26.93641	Val-quantile:51.45217
[11]	Train-quantile:25.75210	Val-quantile:48.71197
[12]	Train-quantile:24.63724	Val-quantile:46.15410
[13]	Train-quantile:23.58743	Val-quantile:43.77402
[14]	Train-quantile:22.59965	Val-quantile:41.56073
[15]	Train-quantile:21.67010	Val-quantile:39.51059
[16]	Train-quantile:20.79593	Val-quantile:37.60501
[17]	Train-quantile:19.97225	Val-quantile:35.83565
[18]	Train-quantile:19.19629	Val-quantile:34.20189
[19]	Train-quantile:18.46553	Val-quantile

### __Lightgbm implementation__

In [9]:
import lightgbm as lgb
importlib.reload(model_utils)

quantiles = [x for x in np.arange(0.1, 1.0, 0.1)]
qr_solar_lightgbm = {}
qr_solar_lightgbm["true"] = feature_engineerer_solar.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr_lightgbm = lgb.LGBMRegressor(objective='quantile', alpha=quantile)
    qr_lightgbm.fit(feature_engineerer_solar.X_train, feature_engineerer_solar.y_train)
    qr_solar_lightgbm[str(quantile)] = qr_lightgbm.predict(feature_engineerer_solar.X_test)

qr_solar_lightgbm_df = pd.DataFrame(qr_solar_lightgbm)
model_utils.pinball_score(qr_solar_lightgbm_df, quantiles=quantiles)

ModuleNotFoundError: No module named 'lightgbm'

In [8]:
quantiles = [x for x in np.arange(0.1, 1.0, 0.1)]
qr_wind_lightgbm = {}
qr_wind_lightgbm["true"] = feature_engineerer_wind.y_test.values
#out_bounds_predictions = np.zeros_like(y_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr_lightgbm_wind = lgb.LGBMRegressor(objective='quantile', alpha=quantile)
    qr_lightgbm_wind.fit(feature_engineerer_wind.X_train, feature_engineerer_wind.y_train)
    qr_wind_lightgbm[str(quantile)] = qr_lightgbm_wind.predict(feature_engineerer_wind.X_test)

qr_wind_lightgbm_df = pd.DataFrame(qr_wind_lightgbm)
model_utils.pinball_score(qr_wind_lightgbm_df, quantiles=quantiles)

NameError: name 'lgb' is not defined